In [47]:
import pandas as pd
base = pd.read_csv('census.csv')
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [48]:
base['income'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [49]:
def converte_classe(rotulo):
    if rotulo == ' >50K':
        return 1
    else:
        return 0

In [50]:
base['income'] = base['income'].apply(converte_classe)

In [51]:
base.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [52]:
base['income'].unique()

array([0, 1])

In [53]:
X = base.drop('income', axis = 1)

In [54]:
X.head()

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loos,hour-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [55]:
y = base['income']

In [56]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int64

In [57]:
type(y)

pandas.core.series.Series

In [ ]:
%matplotlib inline
base.age.hist()

In [58]:
import tensorflow as tf

In [59]:
idade = tf.feature_column.numeric_column('age')

In [60]:
idade_categorica = [tf.feature_column.bucketized_column(idade, boundaries=[20,30,40,50,60,70,80,90])]

In [61]:
print(idade_categorica)

[_BucketizedColumn(source_column=_NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(20, 30, 40, 50, 60, 70, 80, 90))]


In [62]:
X.columns

Index(['age', 'workclass', 'final-weight', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loos', 'hour-per-week', 'native-country'],
      dtype='object')

In [63]:
nome_colunas_categoricas = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex',
                           'native-country']

In [64]:
X['sex'].unique()

array([' Male', ' Female'], dtype=object)

In [65]:
X['workclass'].unique()

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [66]:
X['native-country'].unique()

array([' United-States', ' Cuba', ' Jamaica', ' India', ' ?', ' Mexico',
       ' South', ' Puerto-Rico', ' Honduras', ' England', ' Canada',
       ' Germany', ' Iran', ' Philippines', ' Italy', ' Poland',
       ' Columbia', ' Cambodia', ' Thailand', ' Ecuador', ' Laos',
       ' Taiwan', ' Haiti', ' Portugal', ' Dominican-Republic',
       ' El-Salvador', ' France', ' Guatemala', ' China', ' Japan',
       ' Yugoslavia', ' Peru', ' Outlying-US(Guam-USVI-etc)', ' Scotland',
       ' Trinadad&Tobago', ' Greece', ' Nicaragua', ' Vietnam', ' Hong',
       ' Ireland', ' Hungary', ' Holand-Netherlands'], dtype=object)

In [67]:
colunas_categoricas = [tf.feature_column.categorical_column_with_vocabulary_list(key = c, vocabulary_list=X[c].unique()) for c in nome_colunas_categoricas]

In [68]:
print(colunas_categoricas[7])

_VocabularyListCategoricalColumn(key='native-country', vocabulary_list=(' United-States', ' Cuba', ' Jamaica', ' India', ' ?', ' Mexico', ' South', ' Puerto-Rico', ' Honduras', ' England', ' Canada', ' Germany', ' Iran', ' Philippines', ' Italy', ' Poland', ' Columbia', ' Cambodia', ' Thailand', ' Ecuador', ' Laos', ' Taiwan', ' Haiti', ' Portugal', ' Dominican-Republic', ' El-Salvador', ' France', ' Guatemala', ' China', ' Japan', ' Yugoslavia', ' Peru', ' Outlying-US(Guam-USVI-etc)', ' Scotland', ' Trinadad&Tobago', ' Greece', ' Nicaragua', ' Vietnam', ' Hong', ' Ireland', ' Hungary', ' Holand-Netherlands'), dtype=tf.string, default_value=-1, num_oov_buckets=0)


In [69]:
nome_colunas_numericas = ['final-weight', 'education-num', 'capital-gain', 'capital-loos', 'hour-per-week']

In [70]:
colunas_numericas = [ tf.feature_column.numeric_column(key = c) for c in nome_colunas_numericas ]

In [71]:
print(colunas_numericas[1])

_NumericColumn(key='education-num', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [72]:
colunas = idade_categorica + colunas_categoricas + colunas_numericas

In [73]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3)

In [74]:
X_treinamento.shape
type(X_treinamento)

pandas.core.frame.DataFrame

In [75]:
X_teste.shape
type(y_treinamento)

pandas.core.series.Series

In [76]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento, y = y_treinamento,
                                                        batch_size = 32, num_epochs = None, shuffle = True)

In [77]:
classificador = tf.estimator.LinearClassifier(feature_columns=colunas)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp2faq53dd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6ddb187be0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [78]:
classificador.train(input_fn=funcao_treinamento, steps = 10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp2faq53dd/model.ckpt.
INFO:tensorflow:loss = 22.18071, step = 1
INFO:tensorflow:global_step/sec: 68.9924
INFO:tensorflow:loss = 17426.488, step = 101 (1.451 sec)
INFO:tensorflow:global_step/sec: 187.282
INFO:tensorflow:loss = 8058.8896, step = 201 (0.540 sec)
INFO:tensorflow:global_step/sec: 171.028
INFO:tensorflow:loss = 778.90247, step = 301 (0.582 sec)
INFO:tensorflow:global_step/sec: 175.012
INFO:tensorflow:loss = 38025.14, step = 401 (0.575 sec)
INFO:tensorflow:global_step/sec: 129.978
INFO:tensorflow:loss = 4519.9023, step = 501 (0.766 sec)
INFO:tensorflow:global_step/sec: 148.231
INFO:tensorflow:loss = 3520.4204, step = 601 (0.675 sec)
INFO:tensorflow:global_step/sec: 150.435
INFO:tensorflow:lo

INFO:tensorflow:global_step/sec: 136.181
INFO:tensorflow:loss = 451.7268, step = 8101 (0.741 sec)
INFO:tensorflow:global_step/sec: 138.207
INFO:tensorflow:loss = 509.4805, step = 8201 (0.717 sec)
INFO:tensorflow:global_step/sec: 145.224
INFO:tensorflow:loss = 1497.1812, step = 8301 (0.689 sec)
INFO:tensorflow:global_step/sec: 135.658
INFO:tensorflow:loss = 2188.8726, step = 8401 (0.737 sec)
INFO:tensorflow:global_step/sec: 140.714
INFO:tensorflow:loss = 1314.6506, step = 8501 (0.711 sec)
INFO:tensorflow:global_step/sec: 135.646
INFO:tensorflow:loss = 3720.7456, step = 8601 (0.737 sec)
INFO:tensorflow:global_step/sec: 163.531
INFO:tensorflow:loss = 1097.5215, step = 8701 (0.611 sec)
INFO:tensorflow:global_step/sec: 151.589
INFO:tensorflow:loss = 6187.822, step = 8801 (0.660 sec)
INFO:tensorflow:global_step/sec: 156.86
INFO:tensorflow:loss = 2557.0781, step = 8901 (0.637 sec)
INFO:tensorflow:global_step/sec: 142.219
INFO:tensorflow:loss = 3793.7354, step = 9001 (0.705 sec)
INFO:tensorflo

In [ ]:
funcao_previsao = tf.estimator.inputs.pandas_input_fn(x = X_teste, batch_size = 32, shuffle = False)

In [ ]:
previsoes = classificador.predict(input_fn=funcao_previsao)

In [ ]:
list(previsoes)

In [ ]:
previsoes_final = []
for p in classificador.predict(input_fn=funcao_previsao):
    previsoes_final.append(p['class_ids'])

In [ ]:
previsoes_final

In [ ]:
from sklearn.metrics import accuracy_score
taxa_acerto = accuracy_score(y_teste, previsoes_final)
taxa_acerto